# Improving Performance

## Overview

### Questions

- How can I write custom actions to be as efficient as possible?

### Objectives

- Mention high performance libraries that can help speed up computation
- Demonstrate using the local snapshot API for increased performance.

## Boilerplate Code

In [1]:
import hoomd